# Object References, Mutability, and Recycling
A surprising trait of tuples is revealed: they are immutable but their values may change. This leads to a discussion of shallow and deep copies. References and function parameters are our next theme: the problem with mutable parameter defaults and the safe handling of mutable arguments passed by clients of our functions. <br>

The last sections of the chapter cover garbage collection, the del command, and how to use weak references to 'remember' objects without keeping them alive.

In [1]:
t = (1, 2, [1, 2])

In [5]:
t[0], t[1], t[2]

(1, 2, [1, 2])

In [6]:
import traceback

try:
    t[0] += 1
except:
    traceback.print_exc()

Traceback (most recent call last):
  File "<ipython-input-6-8908e6e22935>", line 4, in <module>
    t[0] += 1
TypeError: 'tuple' object does not support item assignment


In [7]:
import traceback

t[2].append(3)

In [8]:
t

(1, 2, [1, 2, 3])

The usual “variables as boxes” metaphor actually hinders the understanding of reference variables in OO languages. Python variables are like reference variables in Java, so it’s better to think of them as labels attached to objects.

In [9]:
class Gizmo(object):
    def __init__(self):
        print("Gizmo id: %d" % id(self))

x = Gizmo()

Gizmo id: 1701164970336


In [10]:
import traceback

try:
    y = Gizmo() * 10
except:
    traceback.print_exc()

Gizmo id: 1701164972352


Traceback (most recent call last):
  File "<ipython-input-10-b3fd75d698a5>", line 4, in <module>
    y = Gizmo() * 10
TypeError: unsupported operand type(s) for *: 'Gizmo' and 'int'


That proves that this second Gizmo was actually instantiated before the multiplication was attempted. Because variables are mere labels, nothing prevents an object from having several labels assigned to it. When that happens, you have aliasing, our next topic.

## Identity, Equality, and Aliases

In [11]:
charles = {'name': 'Charles L. Dodgson', 'born': 1832}
lewis = charles
lewis is charles

True

In [12]:
id(charles), id(lewis)

(1701164218264, 1701164218264)

In [13]:
lewis['balance'] = 950
charles

{'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}

In [14]:
alex = {'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}
alex == charles    

True

In [16]:
# they are equal, but they are not the same object
# and do not occupy the same space in memory.
alex is charles

False

## Choosing Between == and is
We often care about values and not identities, so == appears more frequently than is in Python code. However, if you are comparing a variable to a singleton, then it makes sense to use is. By far, the most common case is checking whether a variable is bound to None. This is the recommended way to do it:

In [19]:
x = 42
x is None

False

In [20]:
x is not None

True

In [21]:
id(None)

1554687120

The is operator is faster than ==, because it cannot be overloaded, so Python does not have to find and invoke special methods to evaluate it.

## The Relative Immutability of Tuples
Tuples, like most Python collections—lists, dicts, sets, etc.—hold references to objects. If the referenced items are mutable, they may change even if the tuple itself does not.

In [1]:
t = (1, 2, [1, 2])
t

(1, 2, [1, 2])

In [2]:
t[2].append(3)
t

(1, 2, [1, 2, 3])

## Copies Are Shallow by Default
The easiest way to copy a list (or most built-in mutable collections) is to use the builtin constructor for the type itself. For example:

In [3]:
l1 = [3, [55, 44], (7, 8, 9)]
l1

[3, [55, 44], (7, 8, 9)]

In [4]:
l2 = l1
l2 is l1

True

In [5]:
l3 = list(l1)
l3 is l1

False

In [6]:
l3 == l1

True

However, using the constructor or [:] produces a shallow copy (i.e., the outermost container is duplicated, but the copy is filled with references to the same items held by the original container).  This saves memory and causes no problems if all the items are immutable. But if there are mutable items, this may lead to unpleasant surprises.

In [7]:
l1 = [1, 2, {1, 2}]
l1

[1, 2, {1, 2}]

In [8]:
l2 = list(l1)
l2 is l1

False

In [9]:
l2[2] is l1[2]

True

In [10]:
[id(i) for i in l1] # copy is filled with references to the same items

[1529900048, 1529900080, 1893045982600]

In [11]:
[id(i) for i in l2]

[1529900048, 1529900080, 1893045982600]

In [12]:
l1, l2

([1, 2, {1, 2}], [1, 2, {1, 2}])

In [17]:
type(l1[2])

set

In [18]:
l1[2].add(3)

In [19]:
l1

[1, 2, {1, 2, 3}]

In [20]:
l2

[1, 2, {1, 2, 3}]

## Deep and Shallow Copies of Arbitrary Objects
Working with shallow copies is not always a problem, but sometimes you need to make deep copies (i.e., duplicates that do not share references of embedded objects). The copy module provides the deepcopy and copy functions that return deep and shallow copies of arbitrary objects.

In [21]:
l1 = [1, 2, {1, 2}]
l2 = list(l1)

[l1[i] is l2[i] for i in range(3)]

[True, True, True]

In [22]:
import copy
l1 = [1, 2, {1, 2}]
l2 = copy.deepcopy(l1)
[l1[i] is l2[i] for i in range(3)]

[True, True, False]

In [32]:
x = 3
y = 3
x is y

True

In [24]:
x is y

True

In [25]:
id(x), id(y)

(1529900112, 1529900112)

See ingeger objects: https://docs.python.org/3/c-api/long.html <br>
Also here on SO: https://stackoverflow.com/questions/306313/is-operator-behaves-unexpectedly-with-integers <br>
The current implementation keeps an array of integer objects for all integers between -5 and 256, when you create an int in that range you actually just get back a reference to the existing object. So it should be possible to change the value of 1. I suspect the behaviour of Python in this case is undefined.

In [30]:
x = 257
y = 257
x is y

False

In [31]:
id(x), id(y)

(1893049555856, 1893049556112)

In [34]:
x, y = 257, 257 # I'm sure there is something to this.
x is y

True

## Function Parameters as References
The only mode of parameter passing in Python is call by sharing. That is the same mode used in most OO languages, including Ruby, SmallTalk, and Java (this applies to Java reference types; primitive types use call by value). Call by sharing means that each formal parameter of the function gets a copy of each reference in the arguments. 

In [37]:
def f(a, b):
    a += b
    return a


In [38]:
x, y = 1, 2
f(x, y)

3

In [40]:
x, y # x is not changed

(1, 2)

In [41]:
x, y = [1, 2], [3, 4]

f(x, y)

[1, 2, 3, 4]

In [43]:
x, y # x has been changed

([1, 2, 3, 4], [3, 4])

## Mutable Types as Parameter Defaults: Bad Idea
You should avoid mutable objects as default values for parameters.

In [1]:
class HauntedBus(object):
    ''' A bus model haunted by ghost passengers '''
    def __init__(self, passengers=[]):
        self.passengers = passengers

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)
        

In [2]:
bus1 = HauntedBus(['Alice', 'Bill'])
bus1.passengers

['Alice', 'Bill']

In [3]:
bus1.pick('Charlie')
bus1.passengers

['Alice', 'Bill', 'Charlie']

In [4]:
bus1.drop('Alice')
bus1.passengers

['Bill', 'Charlie']

In [5]:
bus2 = HauntedBus([])
bus2.pick('Carrie')
bus2.passengers

['Carrie']

In [6]:
bus3 = HauntedBus()
bus3.passengers

[]

In [7]:
bus3.pick('Dave')
bus2.passengers

['Carrie']

In [8]:
bus2.passengers is bus3.passengers 

False

Hmmm... not quite the same results as the book for this example. <br>
There is another example [here](https://docs.python-guide.org/writing/gotchas/).

In [9]:
def append_to(element, to=[]):
    to.append(element)
    return to


In [10]:
my_list = append_to(12)
print(my_list)

my_other_list = append_to(42)
print(my_other_list)

[12]
[12, 42]


In [11]:
my_list

[12, 42]

In [12]:
my_list is my_other_list

True

To show more of what is at play here...

In [1]:
class MyList(list):
    def __init__(self):
        super(MyList, self).__init__()
        print("Creating an instance of MyList at %d!" % id(self))

    def __del__(self):
        print("Deleting an instance of MyList at %d!" % id(self))


In [2]:
def append_to(element, to=MyList()):
    to.append(element)
    return to


Creating an instance of MyList at 66910392!


In [3]:
print(append_to(12))

[12]


In [4]:
print(append_to(42))

[12, 42]


In [5]:
del append_to

Deleting an instance of MyList at 66910392!


So the list instance is getting instantiated when the function is defined. New calls to the append_to function to not create a new list, they just append to the existing MyList object. The list is held by the function and is only deleted when the function is deleted.

In [2]:
def append_to(element, to=MyList()):
    to.append(element)
    return to

Creating an instance of MyList at 59754440!


In [8]:
append_to.func_defaults

([],)

In [9]:
type(append_to.func_defaults[0])

__main__.MyList

## Defensive Programming with Mutable Parameters
When you are coding a function that receives a mutable parameter, you should carefully consider whether the caller expects the argument passed to be changed. ['Principle of least astonishment'](https://en.wikipedia.org/wiki/Principle_of_least_astonishment), a best practice of interface design. Unless a method is explicitly intended to mutate an object received as argument, you should think twice before aliasing the argument object by simply assigning it to an instance variable in
your class. If in doubt, make a copy. Your clients will often be happier. 

## del and Garbage Collection
The del statement deletes names, not objects. An object may be garbage collected as result of a del command, but only if the variable deleted holds the last reference to the object, or if the object becomes unreachable. In CPython, the primary algorithm for garbage collection is reference counting. Essentially, each object keeps count of how many references point to it. As soon as that refcount reaches zero, the object is immediately destroyed: CPython calls the __del__ method on the object (if defined) and then frees the memory allocated to the object. [See weakref finalize](https://docs.python.org/3/library/weakref.html#weakref.finalize) which is new in Python 3.4 and can simplify object lifecycle management.

In [1]:
import weakref

s1 = {1, 2, 3}
s2 = s1

def bye():
    print('Gone with the wind...')

ender = weakref.finalize(s1, bye) # only available with Python 3
ender.alive

True

In [2]:
del s1

In [3]:
ender.alive

True

In [4]:
s2 = 'spam'

Gone with the wind...


## Weak References
The presence of references is what keeps an object alive in memory. When the reference count of an object reaches zero, the garbage collector disposes of it. But sometimes it is useful to have a reference to an object that does not keep it around longer than necessary. A common use case is a cache. Weak references to an object do not increase its reference count. 

In [1]:
class SomeClass(object):
    def __init__(self, x):
        print("SomeClass instance %d created!" % id(self))
        self.x = x

    def __del__(self):
        print("SomeClass instance %d deleted!" % id(self))


In [2]:
a = SomeClass(7) # reference count is 1

SomeClass instance 2757023064416 created!


In [3]:
b = a  # reference count is now 2

In [4]:
b is a # they both point to the same object in memory

True

In [5]:
import weakref

wref = weakref.ref(a)  # doesn't increase the reference count

In [6]:
wref() is a and a is b # call wref() for a strong reference to the object

True

In [7]:
del b # reference count is now 1

In [8]:
del a # reference count is now zero and and __del__ method  is called

SomeClass instance 2757023064416 deleted!


In [11]:
wref() is None # calls to wref() will now return None

True

Consider using WeakKeyDictionary, WeakValueDictionary, WeakSet, and finalize (which use weak references internally) instead of creating and handling your own weak ref.ref instances by hand. 

## The WeakValueDictionary Skit
The class WeakValueDictionary implements a mutable mapping where the values are weak references to objects.

In [1]:
import weakref

a_set = {0, 1}
wref = weakref.ref(a_set)
wref

<weakref at 0x000002C92B8D8EA8; to 'set' at 0x000002C92B8B9F28>

In [2]:
 wref() is a_set

True

In [3]:
a_set = {2, 3, 4} # original a_set object destroyed

In [4]:
wref() is a_set

False

In [6]:
wref() is None

True

The class WeakValueDictionary implements a mutable mapping where the values are weak references to objects. When a referred object is garbage collected elsewhere in the program, the corresponding key is automatically removed from WeakValueDiction
ary. This is commonly used for caching.

In [7]:
import weakref

class Cheese:
    def __init__(self, kind):
        self.kind = kind
    
    def __repr__(self):
        return 'Cheese(%r)' % self.kind

    
stock = weakref.WeakValueDictionary()
catalog = [Cheese('Red Leicester'), Cheese('Tilsit'),
           Cheese('Brie'), Cheese('Parmesan')]

for cheese in catalog:
    stock[cheese.kind] = cheese

sorted(stock.keys())

['Brie', 'Parmesan', 'Red Leicester', 'Tilsit']

In [8]:
del catalog

In [9]:
 sorted(stock.keys())

['Parmesan']

In [10]:
print(cheese) # the cheese variable is increasing the refcount for Parmesan

Cheese('Parmesan')


In [11]:
del cheese
sorted(stock.keys())

[]

## Limitations of Weak References
Not every Python object may be the target, or referent, of a weak reference. Basic list and dict instances may not be referents, but a plain subclass of either can solve this problem easily...

In [1]:
class MyList(list):
    ''' list subclass whose instances may be weakly referenced '''


In [3]:
import weakref

a_list = MyList(range(10))

# a_list can be the target of a weak reference
wref_to_a_list = weakref.ref(a_list)

In [4]:
wref_to_a_list() is a_list

True

In [5]:
del a_list

In [6]:
print(wref_to_a_list)

<weakref at 0x00000247775C8E08; dead>


In [7]:
wref_to_a_list() is None

True

## Tricks Python Plays with Immutables
A tuple built from another is actually the same exact tuple

In [8]:
t1 = (1, 2, 3)
t2 = tuple(t1)
t2 is t1

True

In [10]:
t3 = t1[:] # no copying here...
t3 is t1

True

An example for numbers and strings...

In [11]:
t1 = (1, 2, 3)
t3 = (1, 2, 3)
t3 is t1

False

In [12]:
s1 = 'ABC'
s2 = 'ABC'
s2 is s1

True